In [ ]:
!pip install timm

In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
import os
from tqdm import tqdm
import timm

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
df = pd.read_csv(r"/kaggle/input/data-split/meta_data_64_16_20.csv")
df

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class FishClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(FishClassifier, self).__init__()
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True)

        in_features = self.efficientnet.classifier.in_features
        self.efficientnet.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.efficientnet(x)

model = FishClassifier()

In [ ]:
model = nn.DataParallel(model)

state_dict = torch.load("/kaggle/input/efficientnet/fish_classifier_EfficientNet-B0.pth", map_location=device)

model.load_state_dict(state_dict)

model = model.module

model.eval()

In [ ]:
class EfficientNetFeatureExtractor(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.features = nn.Sequential(
            base_model.conv_stem,
            base_model.bn1,
            *list(base_model.blocks[:6])
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) 
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x
        
extractor = EfficientNetFeatureExtractor(model.efficientnet)
extractor.eval()
extractor.to(device)
extractor

In [ ]:
base_folder = r"/kaggle/input/manhmeo/GG/xzyx7pbr3w-1"

features = []
labels = []
paths = []
types = []

for i, row in tqdm(df.iterrows(), desc="Feature Extraction"):
    path = os.path.join(base_folder, row["Fish Name"], row["Path"])
    try:
        if not os.path.exists(path):
            path = path.replace('_5', '_0')

        image = Image.open(path).convert("RGB")

        tensor_image = data_transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            features_batch = extractor(tensor_image)

        features.append(features_batch.squeeze(0).cpu().numpy())
        labels.append(row["Label"])
        types.append(row["Type"])
        paths.append(os.path.join(row["Fish Name"], row["Path"]))

    except Exception as e:
        print(f"Error at {path}: {e}")

results = pd.DataFrame({"Path": paths, "Label": labels, "Type": types})
results

In [ ]:
cols = [f"Feature_{i}" for i in range(192)]

df1 = pd.DataFrame(features, columns=cols)
df1

In [ ]:
new_df = pd.concat([results, df1], axis=1)
new_df

In [ ]:
new_df.to_csv(r"/kaggle/working/feature_extraction_EfficientNetB0_Block6.csv", index=False)